In [1]:
import language_tool_python
import pandas as pd

tool = language_tool_python.LanguageTool('en-US', config={'cacheSize': 10000, 'pipelineCaching': True, 'maxCheckThreads': 30, 'warmUp': True})

In [2]:
df = pd.read_csv('wav2vec2-submissions/2337+1781 DSTA raw audiomentations unfreeze val_0.00269 lb 0.01351.csv')

In [3]:
def correct(text):
    match = tool.check(text)
    if match:
        for i, m in enumerate(match):
            if match[i].message == 'Possible spelling mistake found.' and match[i].replacements:
                match[i].replacements[0] = match[i].replacements[0].split()[0]  # prevent it from adding new words
        text = language_tool_python.utils.correct(text, match).upper()
    return text

In [4]:
df['annotation'] = df['annotation'].apply(correct)  # takes 1min

In [5]:
df.to_csv('wav2vec2-submissions/2337+1781 DSTA raw audiomentations unfreeze val_0.00269 lb 0.01351 corrected no word2vec.csv', index=False)

In [ ]:
tool.close()  # doesnt work for me i had to task kill so be careful

In [12]:
text = 'THEY COMPLETE THE CROSSWORD PUZZLE WITHIN FOUR MINUTES WITH NO ERRORSR'
tool.correct(text)

'THEY COMPLETE THE CROSSWORD PUZZLE WITHIN FOUR MINUTES WITH NO ERRORS'